In [25]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

from tqdm.auto import trange
import random
import numpy as np

In [3]:
name_model = "cointegrated/rut5-base-multitask"
model = T5ForConditionalGeneration.from_pretrained(name_model)
tokenizer = T5Tokenizer.from_pretrained(name_model)

In [10]:
def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

Read the list of file lines

In [40]:
def read_list(file_name):
    with open (file_name, 'r', encoding='utf-8') as f:
        line = f.read()   
    return line.split('\n')

In [43]:
Q9 = read_list('Q9.txt')
A9 = read_list('A9.txt')

954 954


In [45]:
QA = []
for i in range(len(Q9)):
    q = Q9[i]
    a = A9[i]
    QA += [[q, a]]

In [46]:
print(len(Q9), len(A9), len(QA))

954 954 954


## Train

In [60]:
pairs = QA

batch_size = 4  # сколько примеров показываем модели за один шаг
report_steps = 100  # раз в сколько шагов печатаем результат
epochs = 6  # сколько раз мы покажем данные модели
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [61]:
model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        # encode the question and answer  
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        # -100 - a special value that allows you to ignore tokens
        y.input_ids[y.input_ids == 0] = -100
        # calculate the loss function
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        # do a step of gradient descent
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))

EPOCH 0


  0%|          | 0/238 [00:00<?, ?it/s]

step 0 loss 3.972487688064575
step 100 loss 3.117737009525299
step 200 loss 3.0479803824424745
EPOCH 1


  0%|          | 0/238 [00:00<?, ?it/s]

step 0 loss 2.9752501964569094
step 100 loss 2.9333178544044496
step 200 loss 2.8076193034648895
EPOCH 2


...

step 0 loss 2.737358478307724
step 100 loss 2.5827356243133544
step 200 loss 2.4595706629753114
EPOCH 4


  0%|          | 0/238 [00:00<?, ?it/s]

step 0 loss 2.511818298101425
step 100 loss 2.476200841665268
step 200 loss 2.475251042842865
EPOCH 5


  0%|          | 0/238 [00:00<?, ?it/s]

step 0 loss 2.506560516357422
step 100 loss 2.469781070947647
step 200 loss 2.33880824804306


In [62]:
new_model_name = 'rut51'  # name of the folder to save
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

('rut51\\tokenizer_config.json',
 'rut51\\special_tokens_map.json',
 'rut51\\spiece.model',
 'rut51\\added_tokens.json')

## Eval the model

In [63]:
model.eval()
def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device)
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [64]:
answer('Спасибо большое что дали настолько много времени, я успешно завершил зачёт')

'Зачет принят.'

In [67]:
answer('Писала Вам по поводу доступа на курсы на openedu. Отправляла две почты, на которые регестрировалась. Доступ так не получила. Окончательная почта, на которую зарегестрировалась - Очень прошу помочь с доступом на курсы!С Уважением,Рягузова МаргаритаМагистр ИТМО, Управление качеством.Отправлено через meizu X8')

'Да, доступ будет.'

In [70]:
answer('Какой период обучения у потока ЦКМ 3? ')

'Ожидайте уведомления о консультации.'

Checking the model

In [74]:
path = 'rut51'
tokenizer51 = T5Tokenizer.from_pretrained(path)
print(type(tokenizer51))

<class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>


In [75]:
model51 = T5ForConditionalGeneration.from_pretrained(path)
print(type(model51))

<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>


In [137]:
model51.eval()
def answer51(x, **kwargs):
    inputs = tokenizer51(x, return_tensors='pt').to(model51.device)
    print(inputs)
    print(inputs['input_ids'].shape)
    with torch.no_grad():
        hypotheses = model51.generate(**inputs, **kwargs)
    return tokenizer51.decode(hypotheses[0], skip_special_tokens=True)

In [138]:
answer51('По предметам модуля цифровая культура записана в поток 4.1, по ошибке зарегистрировалась на экзамен утром 23.09. Хотела бы отменить запись.Афонина Виктория, 309898, группа С4106--')

{'input_ids': tensor([[ 1152,  5713,  1164, 14656,   396,  9278,  5851, 19988,  6331,  2895,
           315,   259, 20218,  1400,   353,   261,   425, 10257,  1291, 16383,
          5822,   310,  2031, 22585,   456, 17418,  3423,  7455,   259, 16572,
           308,  1306,  6404, 12554, 18177,   260,  1602,  9248,  2244, 11522,
          1195,   261,   733,  2660,  2660,   261,   259, 17402,   688,  2309,
          1850,   264,   264,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}
torch.Size([1, 54])


'Зачет принят.'